# Obtaining and Cleaning Data from Flat File, HTML, and API

### Resulting 3 data frames saved in Excel files
 - NFL Games Data.xlsx
 - NFL Season Stats Data.xlsx
 - NFL Game Weather Data.xlsx


**Flat File:** nfl_games.csv (https://www.kaggle.com/toddwfloyd/footballscores/version/1#) 
**Description:** Tabular data representing each NFL game from 1920 to 2017.  
**Variables:**
1.	date (date)
2.	season (integer) - year
3.	neutral (binary) – denotes whether it was a neutral site
4.	playoff (binary) – denotes whether it was a playoff game
5.	team1 (string) – denotes home team
6.	team2 (string) – denotes away team
7.	elo1 (float) – rating for home team
8.	elo2 (float) – rating for away team
9.	score1 (integer) – score for home team
10.	score2 (integer) – score for away team
11.	result1 (float) – denotes whether home team won, 0.5 denotes tie

**Website Data:** https://www.pro-football-reference.com/years/  
**Description:** Adding the year on to the end of this url leads you to a page holding season statistics for each NFL season from 1920 to 2019, but I will go to 2017 (not including the 2017 season).  
**Variables to Scrape:**
1. year - season
2. team
3. wins
4. losses
5. ties
6. win loss perc - win loss percentage
7. points - points scored
8. points opp - points allowed
9. points diff - point differential
10. mov - margin of victory, (point differential / games played)
11. sos total - strength of schedule using Simple Rating System (SRS)
12. srs total - Team quality relative to average based on Simple Rating System, (srs = mov + sos = osrs + dsrs)
13. srs offense - Offense relative to average (osrs)
14. srs defense - Defense relative to average (dsrs)

**API Data:** https://rapidapi.com/awigmore/api/visual-crossing-weather  
**Description:** This api is able to return historical weather data for a specified period of time and City.  
**Variables:**  
1.	City
2.	Date
3.	Min Temperature
4.	Max Temperature
5.	Wind Speed
6.	Wind Gust
7.	Wind Direction
8.	Precipitation
9.	Precipitation Cover
10.	Snow Depth
11.	Visibility
12.	Weather Type  

### Load Data and Packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from pandas.api.types import CategoricalDtype
from plotnine import *
import requests
from datetime import datetime
from io import StringIO
%matplotlib inline

## Flat File Data
- Update games headers to indicate home/away
- Add team name and location data  
    a. Load additional team data CSV file for mapping  
    b. Clean up team data CSV file in preparation for merge  
    c. Merge with games  
- Update winner column to have team values instead of binary and add a loser column
- Rearrange columns
- Update neutral column to be game location
- Update Locations where only States or team names are present
- Add stadium type column for teams with indoor stadiums
- Add location updates by years for Rams and Chargers

In [4]:
games = pd.read_csv("nfl_games.csv")
games.rename(columns={'team1':'home', 'team2': 'away', 'elo1': 'home elo', 'elo2': 'away elo',
                      'elo_prob1': 'home elo prob', 'score1': 'home score', 'score2': 'away score',
                      'result1': 'winner'}, inplace = True)

# Update games headers to indicate home/away
list_of_teams = games['home']
list_of_teams.append(games['away'])

# Add team name and location data
teams = pd.read_csv("https://raw.githubusercontent.com/leesharpe/nfldata/master/data/teams.csv")
teams.drop(['season', 'pff', 'nfl', 'pfr', 'pfflabel', 'fo', 'short_location', 'hyphenated'], axis = 1, inplace=True)
teams.drop_duplicates(subset='team', inplace=True, keep='first')
teams.rename(columns={'full':'home full', 'location': 'home location', 'nickname': 'home nickname'}, inplace=True)

games = pd.merge(games, teams, left_on='home', right_on='team',how='left')
games.drop(['team'], axis=1, inplace=True)

teams.rename(columns={'home full':'away full', 'home location': 'away location',
                      'home nickname': 'away nickname'}, inplace=True)

games = pd.merge(games, teams, left_on='away', right_on='team',how='left')
games.drop(['team'], axis=1, inplace=True)

# Update winner column to have team values instead of binary and add a loser column
games['loser'] = games['winner'] # creates a new column for loser with placeholder winner values for now

for i in range(len(games['winner'])):
    val = games['winner'][i]
    if val == 1:
        games['winner'][i] = games['home'][i]
        games['loser'][i] = games['away'][i]
    elif val == 0:
        games['winner'][i] = games['away'][i]
        games['loser'][i] = games['home'][i]
    else:
        games['winner'][i] = 'NaN'
        games['loser'][i] = 'NaN'

# Rearrange columns
games = games[['date', 'season', 'neutral', 'playoff', 'home', 'away', 'home elo',
       'away elo', 'home elo prob', 'home score', 'away score', 'winner', 'loser',
       'home full', 'away full', 'home location', 'away location', 'home nickname', 
        'away nickname']]

# Update neutral column to be game location
for i in range(len(games['neutral'])):
    val = games['neutral'][i]
    if val == 1:
        games['neutral'][i] = 'neutral'
    elif val == 0:
        games['neutral'][i] = games['home location'][i]
    else:
        print('Error')

games = games.rename(columns={'neutral':'game location'})

# Update Locations where only States or team names are present
games['game location'] = games['game location'].replace(['New York Giants', 'New York Jets'], 'East Rutherford')
games['home location'] = games['home location'].replace(['New York Giants', 'New York Jets'], 'East Rutherford')
games['away location'] = games['away location'].replace(['New York Giants', 'New York Jets'], 'East Rutherford')

games['game location'] = games['game location'].replace('Minnesota', 'Minneapolis')
games['home location'] = games['home location'].replace('Minnesota', 'Minneapolis')
games['away location'] = games['away location'].replace('Minnesota', 'Minneapolis')

games['game location'] = games['game location'].replace('Arizona', 'Glendale Arizona')
games['home location'] = games['home location'].replace('Arizona', 'Glendale Arizona')
games['away location'] = games['away location'].replace('Arizona', 'Glendale Arizona')

games['game location'] = games['game location'].replace('New England', 'Foxborough Massachusetts')
games['home location'] = games['home location'].replace('New England', 'Foxborough Massachusetts')
games['away location'] = games['away location'].replace('New England', 'Foxborough Massachusetts')

games['game location'] = games['game location'].replace('Tennessee', 'Nashville Tennessee')
games['home location'] = games['home location'].replace('Tennessee', 'Nashville Tennessee')
games['away location'] = games['away location'].replace('Tennessee', 'Nashville Tennessee')

games['game location'] = games['game location'].replace('Carolina', 'Charlotte North Carolina')
games['home location'] = games['home location'].replace('Carolina', 'Charlotte North Carolina')
games['away location'] = games['away location'].replace('Carolina', 'Charlotte North Carolina')

games['game location'] = games['game location'].replace('Tampa Bay', 'Tampa Florida')
games['home location'] = games['home location'].replace('Tampa Bay', 'Tampa Florida')
games['away location'] = games['away location'].replace('Tampa Bay', 'Tampa Florida')

games['game location'] = games['game location'].replace('Los Angeles Chargers', 'Los Angeles')
games['home location'] = games['home location'].replace('Los Angeles Chargers', 'Los Angeles')
games['away location'] = games['away location'].replace('Los Angeles Chargers', 'Los Angeles')

# Add stadium type column for teams with indoor stadiums
games['stadium type'] = games['game location']
for i in range(len(games['game location'])):
    val = games['game location'][i]
    if val in ['Detroit', 'New Orleans', 'Minneapolis', 'Dallas', 'Indianapolis',
               'Atlanta', 'Houston', 'Glendale Arizona']:
        games['stadium type'][i] = 'indoor'
    elif type(val) is float:
        games['stadium type'][i] = 'NaN'
    else:
        games['stadium type'][i] = 'outdoor'
        
# Add location updates by years for Rams and Chargers
for i in range(len(games['season'])):
    
    if games['home'][i] == 'LAR':
        if games['season'][i] >= 1936 and games['season'][i] <= 1945:
            games['game location'][i] = 'Cleveland Ohio'
            games['home location'][i] = 'Cleveland Ohio'
        elif games['season'][i] >= 1946 and games['season'][i] < 1995:
            games['game location'][i] = 'Los Angeles'
            games['home location'][i] = 'Los Angeles'
        elif games['season'][i] >= 1996 and games['season'][i] <= 2015:
            games['game location'][i] = 'Saint Louis Missouri'
            games['home location'][i] = 'Saint Louis Missouri'
        elif games['season'][i] > 2015 and games['season'][i] <= 2020:
            games['game location'][i] = 'Los Angeles'
            games['home location'][i] = 'Los Angeles'
            
    if games['away'][i] == 'LAR':
        if games['season'][i] >= 1936 and games['season'][i] <= 1945:
            games['away location'][i] = 'Cleveland Ohio'
        elif games['season'][i] >= 1946 and games['season'][i] < 1995:
            games['away location'][i] = 'Los Angeles'
        elif games['season'][i] >= 1996 and games['season'][i] <= 2015:
            games['away location'][i] = 'Saint Louis Missouri'
        elif games['season'][i] > 2015 and games['season'][i] <= 2020:
            games['away location'][i] = 'Los Angeles'
            
    if games['home'][i] == 'LAC':
        if games['season'][i] == 1960:
            games['game location'][i] = 'Los Angeles'
            games['home location'][i] = 'Los Angeles'
        elif games['season'][i] > 1960 and games['season'][i] < 2017:
            games['game location'][i] = 'San Diego'
            games['home location'][i] = 'San Diego'
        elif games['season'][i] >= 2017:
            games['game location'][i] = 'Los Angeles'
            games['home location'][i] = 'Los Angeles'
            
    if games['away'][i] == 'LAC':
        if games['season'][i] == 1960:
            games['away location'][i] = 'Los Angeles'
        elif games['season'][i] > 1960 and games['season'][i] < 2017:
            games['away location'][i] = 'San Diego'
        elif games['season'][i] >= 2017:
            games['away location'][i] = 'Los Angeles'

C:\Users\ssear\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ssear\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\ssear\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ssear\anaconda3\lib\site-packages\ipykernel

In [5]:
# Save Dataframe to Excel so I don't need to re-run scripts
games.to_excel('NFL Games Data.xlsx', sheet_name='Data')

## Web Data

In [6]:
years = games["season"].unique()

# Create Dataframe to store captured data
web_data = pd.DataFrame(columns = ['year','team','wins','losses','ties','win loss perc','points',
                                   'points opp','points diff','mov','sos total','srs total',
                                   'srs offense','srs defense'])

for year in years:
    
    # Get html from url for year
    my_url = "https://www.pro-football-reference.com/years/{}/".format(year)
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")
    
    # Get tables to extract data from
    afc_nfc_tables = page_soup.findAll("table", {"class":"sortable stats_table"})

    # Parse conference table(s)
    for conf in range(len(afc_nfc_tables)):
        conference_table_rows = afc_nfc_tables[conf].tbody.findAll("tr")

        # Parse row of conference table
        for row in range(len(conference_table_rows)):

            # 'right left' row denotes the NFL Division and must be skipped
            if len(conference_table_rows[row].findAll("td", {"class":"right left"})) == 0:

                # Capture data for one row
                team_name = conference_table_rows[row].a.text
                wins = conference_table_rows[row].findAll("td", {"data-stat":"wins"})[0].text
                losses = conference_table_rows[row].findAll("td", {"data-stat":"losses"})[0].text

                # One year did not have ties
                if len(conference_table_rows[row].findAll("td", {"data-stat":"ties"})) == 0:
                    ties = 0
                else:
                    ties = conference_table_rows[row].findAll("td", {"data-stat":"ties"})[0].text
                
                # Capture data for one row (continued)
                win_loss_perc = conference_table_rows[row].findAll("td", {"data-stat":"win_loss_perc"})[0].text
                points = conference_table_rows[row].findAll("td", {"data-stat":"points"})[0].text
                points_opp = conference_table_rows[row].findAll("td", {"data-stat":"points_opp"})[0].text
                points_diff = conference_table_rows[row].findAll("td", {"data-stat":"points_diff"})[0].text
                mov = conference_table_rows[row].findAll("td", {"data-stat":"mov"})[0].text
                sos_total = conference_table_rows[row].findAll("td", {"data-stat":"sos_total"})[0].text
                srs_total = conference_table_rows[row].findAll("td", {"data-stat":"srs_total"})[0].text
                srs_offense = conference_table_rows[row].findAll("td", {"data-stat":"srs_offense"})[0].text
                srs_defense = conference_table_rows[row].findAll("td", {"data-stat":"srs_defense"})[0].text


                # Add captured row to dataframe
                new_row = {'year':year, 'team':team_name, 'wins':wins, 'losses':losses, 'ties':ties,
                           'win loss perc':win_loss_perc, 'points':points, 'points opp':points_opp,
                           'points diff':points_diff, 'mov': mov, 'sos total': sos_total, 'srs total':srs_total,
                           'srs offense': srs_offense, 'srs defense':srs_defense}
                web_data = web_data.append(new_row, ignore_index=True)
                
# Convert number columns to numeric
cols = ['year', 'wins','losses','ties', 'win loss perc','points', 'points opp','points diff',
         'mov','sos total','srs total', 'srs offense','srs defense']

web_data[cols] = web_data[cols].apply(pd.to_numeric)

In [7]:
# Save Dataframe to Excel so I don't need to re-run scripts
web_data.to_excel('NFL Season Stats Data.xlsx', sheet_name='Data')

## API Data

In [380]:
# Get array of game locations to iterate through
game_locations = games['game location'].unique()

# Get start and end dates and convert them into correct format for API
end_date = games['date'][len(games['date'])-1]
end_date = str(datetime.strptime(end_date, '%m/%d/%Y').date())
start_date = games['date'][0]
start_date = str(datetime.strptime(start_date, '%m/%d/%Y').date())
print(start_date, end_date)

1920-09-26 2017-02-05


In [381]:
# Year intervals to meet max row requirements of API
intervals = [(1961,1980),(1981,2000),(2001,2017)]

# Empty list of dataframes to concat later
df_list = []

# Response counter
resp_cnt = 0

# Loop through game locations, pulling weather data back for each location
for location in game_locations:
    for interval in intervals:
        start = str(interval[0])
        end = str(interval[1])
        if type(location) is not float and location != 'neutral':
            url = "https://visual-crossing-weather.p.rapidapi.com/history"

            querystring = {"dayStartTime":"12:00:00","contentType":"csv","dayEndTime":"23:00:00",
                           "shortColumnNames":"false","startDateTime":start+"-01-01T00:00:00",
                           "aggregateHours":"24","location":location+" USA",
                           "endDateTime":end+"-12-31T00:00:00","unitGroup":"us"}
            
            # Creds removed
            headers = {
                'x-rapidapi-host': "", 
                'x-rapidapi-key': ""
                }
            
            response = requests.request("GET", url, headers=headers, params=querystring)
            resp_cnt = resp_cnt + 1
            temp_df = pd.read_csv(StringIO(response.text))

            df_list.append(temp_df)

In [382]:
# Concat dataframes into one
weather_data =  pd.concat(df_list, ignore_index=True)

In [383]:
# Checking to see if Addresses were correct
weather_data['Resolved Address'].unique()

array(['Chicago, IL, United States', 'Glendale, AZ, United States',
       'Green Bay, WI, United States',
       'East Rutherford, NJ, United States', 'Detroit, MI, United States',
       'Washington, DC, United States', 'Pittsburgh, PA, United States',
       'Philadelphia, PA, United States', 'Cleveland, OH, United States',
       'San Francisco, CA, United States',
       'Los Angeles, CA, United States',
       'Indianapolis, IN, United States', 'Foxborough, MA, United States',
       'Oakland, CA, United States', 'Nashville, TN, United States',
       'Buffalo, NY, United States', 'Dallas, TX, United States',
       'Kansas City, MO, United States', 'Denver, CO, United States',
       'Minneapolis, MN, United States', 'San Diego, CA, United States',
       'Miami, FL, United States', 'Atlanta, GA, United States',
       'New Orleans, LA, United States', 'Cincinnati, OH, United States',
       'Seattle, WA, United States', 'Tampa, FL, United States',
       'Jacksonville, FL, Unit

In [384]:
# Function for getting dates in the same format
def convert_date(date):
    return str(datetime.strptime(date, '%m/%d/%Y').date())

In [385]:
# Convert the dates with function
weather_data['Date time'] = weather_data['Date time'].apply(convert_date)

In [386]:
weather_data['Date time']

0         1961-01-01
1         1961-01-02
2         1961-01-03
3         1961-01-04
4         1961-01-05
             ...    
687022    2017-12-27
687023    2017-12-28
687024    2017-12-29
687025    2017-12-30
687026    2017-12-31
Name: Date time, Length: 687027, dtype: object

In [387]:
# Filter out rows in weather data where for dates there's no game
weather_data = weather_data[weather_data['Date time'].isin(loc_dates['date'])]
weather_data.shape

(77880, 25)

In [388]:
# Delete the ' USA' part of the location in the weather data df
weather_data['Address'].replace(regex=[' USA'], value = '', inplace=True)

In [389]:
# Save Dataframe to Excel so I don't need to re-run scripts
weather_data.to_excel('NFL Game Weather Data.xlsx', sheet_name='Data')